<a href="https://colab.research.google.com/github/JordanDSIA/2025_UTEC_PROGRAMACION_101/blob/main/Practica%2002%20-%20IA%20Fundamental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar Librerias

In [1]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from matplotlib import pyplot as plt
import time # Importar para medir el tiempo de ejecución

Parámetros del Problema, Nodos y Matriz de Distancias

In [7]:
N = 6       # Cantidad de CLIENTES
N_VEHICLES = 3 # Cantidad de vehículos

# Nodos Generados = N clientes + 1 depósito
positions = np.random.rand(N + 1, 2)

# Calcular la matriz de distancias entre todos los nodos
distances = squareform(pdist(positions, 'euclidean'))

print("Número de clientes:", N)
print("Número total de nodos (1 Deposito + Clientes):", N + 1)
print("Número de vehículos:", N_VEHICLES)
print("\nPosiciones de los nodos (Depósito en índice 0):")
print(positions[:5], "...")
print("\nMatriz de distancias:")
print(distances[:5, :5], "...")

Número de clientes: 6
Número total de nodos (1 Deposito + Clientes): 7
Número de vehículos: 3

Posiciones de los nodos (Depósito en índice 0):
[[0.70627556 0.70196547]
 [0.85381628 0.24010737]
 [0.02922538 0.48728715]
 [0.20907495 0.74452984]
 [0.12899056 0.35140207]] ...

Matriz de distancias:
[[0.         0.4848517  0.71027018 0.49901922 0.67539076]
 [0.4848517  0.         0.86084145 0.81861677 0.73332042]
 [0.71027018 0.86084145 0.         0.31387843 0.16857594]
 [0.49901922 0.81861677 0.31387843 0.         0.40120189]
 [0.67539076 0.73332042 0.16857594 0.40120189 0.        ]] ...


Fitness(Función Objetivo) , Generación de Vecinos y Aceptación

In [ ]:
def fitness(sol):
    distance = 0
    for i in range(len(sol) - 1):
        distance += distances[sol[i]][sol[i+1]]
    # Para VRP con depósitos explícitos en la ruta, distances[sol[-1]][sol[0]] y distances[0][0] que es 0.
    distance += distances[sol[-1]][sol[0]]
    return distance

def get_neighbors(sol):
    neighbors = []
    num_elements = len(sol) # Longitud de la secuencia VRP (clientes + depósitos)
    for i in range(num_elements - 1):
        for j in range(i + 1, num_elements):
            new_sol = sol.copy()
            new_sol[i], new_sol[j] = new_sol[j], new_sol[i]
            neighbors.append(new_sol)
    return neighbors

def random_neighbor(sol):
    num_elements = len(sol) # Longitud de la secuencia VRP (clientes + depósitos)
    i, j = np.random.choice(num_elements, 2, replace=False)
    new_sol = sol.copy()
    new_sol[i], new_sol[j] = new_sol[j], new_sol[i]
    return new_sol

def accept(delta, T):
    if delta < 0:
        return True
    else:
        r = np.random.rand()
        return r < np.exp(-delta/T)

Algoritmos de Búsqueda Local

In [ ]:
def hill_climbing(initial_solution):
    current_solution = initial_solution.copy()
    while True:
        neighbors = get_neighbors(current_solution)
        best_neighbor = min(neighbors, key=fitness)
        if fitness(best_neighbor) < fitness(current_solution):
            current_solution = best_neighbor
        else:
            return current_solution

def simulated_annealing(T_max, T_min, cooling_rate, initial_solution):
    T = T_max
    x = initial_solution.copy()
    E = fitness(x)
    while (T > T_min):
        x_new = random_neighbor(x)
        E_new = fitness(x_new)
        delta = E_new - E
        if accept(delta, T):
            x = x_new
            E = E_new
        T = T * cooling_rate
    return x

Función de Ploteo